In [1]:
import os
import shutil
import stat
import seaborn as sns
import collections
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.image as img
import random
import cv2
import PIL
import matplotlib.pyplot as plt
import matplotlib.image as img
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from skimage.io import imread
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models

In [2]:
# Method to generate directory-file map. 
def gen_dir_file_map(path):
    dir_files = defaultdict(list)
    with open(path, 'r') as txt:
        files = [i.strip() for i in txt.readlines()]
        for f in files:
            dir_name, id = f.split('/')
            dir_files[dir_name].append(id + '.jpg')
    return dir_files

# Method to recursively copy a directory.  
def copytree(source, target, symlinks=False, ignore=None):
    if not os.path.exists(target):
        os.makedirs(target)
        shutil.copystat(source, target)
    data = os.listdir(source)
    if ignore:
        exclude = ignore(source, data)
        data = [x for x in data if x not in exclude]
    for item in data:
        src = os.path.join(source, item)
        dest = os.path.join(target, item)
        if symlinks and os.path.islink(src):
            if os.path.lexists(dest):
                os.remove(dest)
            os.symlink(os.readlink(src), dest)
            try:
                st = os.lstat(src)
                mode = stat.S_IMODE(st.st_mode)
                os.lchmod(dest, mode)
            except:
                pass
        elif os.path.isdir(src):
            copytree(src, dest, symlinks, ignore)
        else:
            shutil.copy2(src, dest)

# Train files to ignore. 
def ignore_train(d, filenames):
    subdir = d.split('/')[-1]
    train_dir_files = gen_dir_file_map("C:/Users/kurt_/Desktop/food-101/meta/train.txt")
    to_ignore = train_dir_files[subdir]
    return to_ignore

# Test files to ignore.    
def ignore_test(d, filenames):
    subdir = d.split('/')[-1]
    test_dir_files = gen_dir_file_map("C:/Users/kurt_/Desktop/food-101/meta/test.txt")
    to_ignore = test_dir_files[subdir]
    return to_ignore

# Method to generate train-test files. 
def gen_train_test_split(path_to_imgs="C:/Users/kurt_/Desktop/food-101/images", target_path="C:/Users/kurt_/Desktop/food-101", meta_path="C:/Users/kurt_/Desktop/food-101/meta"):
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    train_path = os.path.join(target_path, 'train')
    test_path = os.path.join(target_path, 'test')
    if not os.path.exists(train_path):
        os.makedirs(train_path)
    if not os.path.exists(test_path):
        os.makedirs(test_path)
    copytree(path_to_imgs, train_path, ignore=ignore_test)
    copytree(path_to_imgs, test_path, ignore=ignore_train)

# Method to load and resize images.  
def load_images(path_to_imgs, image_size=(331, 331)):
    resize_count = 0
    invalid_count = 0
    all_imgs = []
    all_classes = []

    for subdir in os.listdir(path_to_imgs):
        classN = subdir
        subdir_path = os.path.join(path_to_imgs, subdir)
        for img_name in os.listdir(subdir_path):
            img_path = os.path.join(subdir_path, img_name)
            img_arr = cv2.imread(img_path)
            img_arr_rs = cv2.resize(img_arr, image_size, interpolation=cv2.INTER_AREA)
            resize_count += 1
            im_rgb = cv2.cvtColor(img_arr_rs, cv2.COLOR_BGR2RGB)
            all_imgs.append(im_rgb)
            all_classes.append(classN)
  
    return np.array(all_imgs), np.array(all_classes)


In [3]:
# Method to load train-test files.
def load_train_test_data(path_to_train_imgs, path_to_test_imgs):
    X_train, y_train = load_images(path_to_train_imgs)
    X_test, y_test = load_images(path_to_test_imgs)
    return X_train, y_train, X_test, y_test

# Generate train-test files. 
if not os.path.isdir("C:/Users/kurt_/Desktop/food-101/test") and not os.path.isdir("C:/Users/kurt_/Desktop/food-101/train"):
    gen_train_test_split()  
    len_train = len(os.listdir("C:/Users/kurt_/Desktop/food-101/train"))
    len_test = len(os.listdir("C:/Users/kurt_/Desktop/food-101/test"))
    print(len_train, len_test)
else:
    print('train and test folders already exist.')
    len_train = len(os.listdir("C:/Users/kurt_/Desktop/food-101/train"))
    len_test = len(os.listdir("C:/Users/kurt_/Desktop/food-101/test"))
    print(len_train, len_test)

train and test folders already exist.
101 101


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LambdaCallback
from tensorflow.keras import regularizers

import tensorflow.keras.backend as K

# Clear Keras session
K.clear_session()

# Define constants
n_classes = 101
batch_size = 16
width, height = 331, 331
train_data = r"C:\Users\kurt_\Desktop\food-101\train"
test_data = r"C:\Users\kurt_\Desktop\food-101\test"
train_samples = 75750
test_samples = 25250

# Data generators
train_data_gen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_data_gen = ImageDataGenerator(rescale=1. / 255)
train_gen = train_data_gen.flow_from_directory(train_data, target_size=(height, width), batch_size=batch_size, class_mode='categorical')
test_gen = test_data_gen.flow_from_directory(test_data, target_size=(height, width), batch_size=batch_size, class_mode='categorical')

# Model
inception = InceptionV3(weights='imagenet', include_top=False)
layer = inception.output
layer = GlobalAveragePooling2D()(layer)
layer = Dense(128, activation='relu')(layer)
layer = Dropout(0.2)(layer)
predictions = Dense(n_classes, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(layer)
model = Model(inputs=inception.input, outputs=predictions)

# Compile model
optimizer = SGD(momentum=0.9)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpointer = ModelCheckpoint(filepath='best_model_101class_inceptionv3den3.hdf5', save_best_only=True)
csv_logger = CSVLogger('history_101class_inceptionv3den3.log')

# Progress print callback
def print_progress(epoch, logs):
    print("Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}, Val Loss: {:.4f}, Val Accuracy: {:.4f}".format(
        epoch+1, logs['loss'], logs['accuracy'], logs['val_loss'], logs['val_accuracy']))

progress_printer = LambdaCallback(on_epoch_end=print_progress)

# Train the model
history_101class_inceptionv3den3 = model.fit(train_gen, 
                             steps_per_epoch=train_samples // batch_size, 
                             validation_data=test_gen, 
                             validation_steps=test_samples // batch_size, 
                             epochs=35, 
                             callbacks=[csv_logger, checkpointer, progress_printer])

# Save the model
model.save('model_trained_101class_inceptionv3den3.hdf5')


Found 101000 images belonging to 101 classes.
Found 101000 images belonging to 101 classes.
Epoch 1/35
4734/4734 [==============================] - 1541s 323ms/step - loss: 3.1398 - accuracy: 0.2998 - val_loss: 3.0156 - val_accuracy: 0.3599
Epoch 2/35
4734/4734 [==============================] - 1496s 316ms/step - loss: 2.3717 - accuracy: 0.4710 - val_loss: 2.3499 - val_accuracy: 0.5026
Epoch 3/35
4734/4734 [==============================] - 1479s 312ms/step - loss: 2.0716 - accuracy: 0.5485 - val_loss: 1.9545 - val_accuracy: 0.5899
Epoch 4/35
4734/4734 [==============================] - 1478s 312ms/step - loss: 1.8694 - accuracy: 0.5985 - val_loss: 1.9194 - val_accuracy: 0.5945
Epoch 5/35
4734/4734 [==============================] - 1478s 312ms/step - loss: 1.7182 - accuracy: 0.6351 - val_loss: 1.6078 - val_accuracy: 0.6704
Epoch 6/35
4734/4734 [==============================] - 1505s 318ms/step - loss: 1.6082 - accuracy: 0.6626 - val_loss: 1.2770 - val_accuracy: 0.7441
Epoch 7/35
473

In [5]:
model_path = r"C:\Users\kurt_\best_model_101class_inceptionv3den3.hdf5"
test_data_path = r"C:\Users\kurt_\Desktop\food-101\test"

In [6]:
# Performans metriklerini hesaplamak için fonksiyon tanımla
def calculate_metrics(model, test_gen):
    # Modelden tahminler yap
    predictions = model.predict(test_gen)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Gerçek sınıfları al
    true_classes = test_gen.classes
    
    # Sınıf etiketlerini al
    class_labels = list(test_gen.class_indices.keys())

    # Doğruluk, hassasiyet, geri çağırma ve F1 puanını hesapla
    accuracy = accuracy_score(true_classes, predicted_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    f1 = f1_score(true_classes, predicted_classes, average='weighted')

    return accuracy, precision, recall, f1

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Boyutları tanımla
height, width = 331, 331

# Batch boyutunu tanımla
batch_size = 16

# Test veri üreteci oluştur
test_data_gen = ImageDataGenerator(rescale=1. / 255)
test_gen = test_data_gen.flow_from_directory(test_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical', shuffle=False)

# Modeli yükle
model = load_model(model_path, compile=False)

# Performans metriklerini hesapla
accuracy, precision, recall, f1_score = calculate_metrics(model, test_gen)

# Hesaplanan metrikleri yazdır
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Found 101000 images belonging to 101 classes.
6313/6313 [==============================] - 405s 64ms/step
Accuracy: 0.9414950495049504
Precision: 0.9454587868753546
Recall: 0.9414950495049504
F1 Score: 0.9419139984545255
